In [1]:
# Imports

import pandas as pd
import json
import sklearn
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV, StratifiedShuffleSplit
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA, PCA, SparsePCA
from sklearn.feature_selection import mutual_info_classif, f_classif, SelectKBest, chi2, SelectPercentile
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer



In [2]:
# Datafile from Jedwin's feature extraction
from scipy import sparse
temp_sparse_ = sparse.load_npz( "/content/domain1_X_y_csr.npz" )
temp_loaded = temp_sparse_.toarray()
n_samples, n_features = temp_loaded.shape
n_features -= 1 #Since the last column is actually the label

X = temp_loaded[:,:n_features]
y = temp_loaded[:,n_features]
del temp_loaded, temp_sparse_

In [3]:
selector_percentile = SelectPercentile( f_classif, percentile = 80 )
X_new = selector_percentile.fit_transform(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   22  131  166  211  212  213  214
  242  248  252  254  261  262  263  285  298  318  344  383  387  437
  450  460  463  469  482  483  500  518  538  539  540  541  542  544
  552  555  556  557  562  589  605  606  619  626  634  647  650  656
  661  667  668  682  725  726  729  734  790  843  844  860  880  887
  912  918  934  947  962  985  986  987  991 1003 1027 1050 1071 1108
 1109 1135 1136 1139 1140 1148 1149 1173 1174 1175 1187 1194 1199 1216
 1243 1300 1457 1559 1592 1635 1644 1688 1729 1749 1750 1803 1831 1847
 1848 1894 1897 1953 1959 1964 1973 2001 2031 2038 2042 2043 2063 2070
 2082 2244 2262 2267 2316 2375 2376 2427 2430 2451 2471 2478 2512 2524
 2539 2565 2718 2748 2761 2771 2817 2818 2828 2833 2847 2972 2985 3065
 3131 3132 3139 3189 3190 3296

In [4]:
# SVM grid search for params
C_range = np.logspace(1, 3, 3)
gamma_range = np.logspace(-1, 2, 4)
param_grid = dict(kernel=['linear', 'rbf'], C=C_range, gamma=gamma_range)

grid = GridSearchCV(SVC(), param_grid=param_grid, refit = True, verbose = 3)
grid.fit(X_new, y)

print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))


Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.772 total time= 1.7min
[CV 2/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.770 total time=  58.2s
[CV 3/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.744 total time=  58.3s
[CV 4/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.761 total time= 1.4min
[CV 5/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.755 total time=  56.9s
[CV 1/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.788 total time= 1.3min
[CV 2/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.779 total time= 1.3min
[CV 3/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.776 total time= 1.2min
[CV 4/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.787 total time= 1.3min
[CV 5/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.775 total time= 1.2min
[CV 1/5] END ..C=10.0, gamma=1.0, kernel=linear;, score=0.772 total time= 1.7min
[CV 2/5] END ..C=10.0, gamma=1.0, kernel=linear

In [8]:
# The best parameters are {'C': 100.0, 'gamma': 1.0, 'kernel': 'rbf'} with a score of 0.82
svm_domain1 = SVC(C=100.00, gamma=1.0, kernel='rbf')
svm_domain1.fit(X_new, y)

SVC(C=100.0, gamma=1.0)

In [10]:
# Export model
import pickle as pkl
with open("svm_domain1.mdl", "wb") as writefile:
    pkl.dump([svm_domain1, selector_percentile], writefile )